In [1]:
import numpy as np
import time
import scipy.stats as stats
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
from scipy import stats
from sklearn.metrics import log_loss
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
import matplotlib.pyplot as plt
from sklearn.metrics import (brier_score_loss, precision_score, recall_score,
                             f1_score)
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import RandomOverSampler

In [2]:
data = pd.read_sas('/home/jallen17/Downloads/wnv_2245new.sas7bdat') 

In [3]:
x_selected = data[data.drop(columns=['wnvbinary','yrweeks','yrwksfid','yr_hexid','year']).columns[[7,8,9,10,15,16,17,20]]]
y = data['wnvbinary'].values 
y_selected = y

In [4]:
def model_RF_test(model_RF, dataX, dataY):
    print("Model performance")
    predict_data = model_RF.predict_proba(dataX)
    
    # Some stats
    print("Feature Importantce : ")
    print(model_RF.feature_importances_)
    print("Total number of WNV occurence in test set : " + str(len(dataY[dataY > 0])))
    
    print("Number of WNV occurence the model is able to capture in test set:" + str(dataY[np.where(predict_data[:,1]  > 0)].sum()))
    
    print("Expected number of WNV occurence of models aussume prediction is normally distributed : " + str(predict_data[:,1].mean() * len(predict_data)))
    print("Log loss : " + str(log_loss(dataY,predict_data)))
    
    print("This is to test the performance of random forest model, ideally, the logloss is low and also it is able to capture most of the WNV occurence, and also it is neither over predicting " +  
          "nor under predicting")
    print("AUC : " + str( roc_auc_score(dataY,predict_data[:,1])))
    
    return None # Check how many wnv it predicts

In [5]:
trainX_sel, testX_sel, trainY_sel, testY_sel = train_test_split(x_selected.values, y_selected, test_size = 0.2, shuffle = True) # CV

In [6]:

rus = RandomOverSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(trainX_sel,trainY_sel)
time_start = time.time()
model_RF1 = RandomForestClassifier(n_estimators=500,
                                 n_jobs = -1,
                               max_features='sqrt',
                                 max_depth= None,
                                 bootstrap=True,
                                   min_samples_leaf= 2,
                                   class_weight='balanced'
                                 ) # Use undersampling to see if it worked
model_RF1.fit(X_resampled, y_resampled)
print("time consumed:", time.time() - time_start) 

time consumed: 926.9206185340881


In [7]:
model_RF_test(model_RF1,testX_sel,testY_sel)

Model performance
Feature Importantce : 
[0.11520061 0.1241793  0.1354357  0.06621842 0.15306396 0.18880771
 0.14234262 0.07475167]
Total number of WNV occurence in test set : 186
Number of WNV occurence the model is able to capture in test set:109.0
Expected number of WNV occurence of models aussume prediction is normally distributed : 286.5051712422878
Log loss : 0.0121963712878016
This is to test the performance of random forest model, ideally, the logloss is low and also it is able to capture most of the WNV occurence, and also it is neither over predictingnor under predicting
AUC : 0.724793246605591
